In [ ]:
import os

hdfs_fs = 'hdfs://hadoop-master.hadoop-domain.default-tenant.svc.cluster.local:9000'
v3io_fs =  os.getenv('V3IO_HOME_URL')

print(f"HDFS: {hdfs_fs}")
print(f"V3IO: {v3io_fs}")

In [ ]:
krb5_conf_dir = '/User/conf/kerberos'
hadoop_conf_dir = '/User/conf/hadoop'
krb5_cc_name = 'FILE:' + os.path.join(krb5_conf_dir,'krb5_ccache')
krb5_config_file = os.path.join(krb5_conf_dir,'krb5.conf')
krb5_keytab_file = os.path.join(krb5_conf_dir,'krb5.keytab')

print(f"KRB5CCNAME: {krb5_cc_name}")
print(f"HADOOP_CONF_DIR: {hadoop_conf_dir}")
print(f"KRB5_CONFIG: {krb5_config_file}")
print(f"JVM config: {jvm_config_option}")

In [ ]:
# These should be pre-configured in the Jupyter service configuration. Use this only if they're not.
# os.environ['KRB5CCNAME'] = krb5_cc_name
# os.environ['HADOOP_CONF_DIR'] = hadoop_conf_dir

In [ ]:
!kinit -k -t /User/conf/kerberos/krb5.keytab hdfs/hadoop-master.hadoop-domain.default-tenant.svc.cluster.local@EXAMPLE.COM
!klist

In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.appName("Example") \
    .master('k8s://https://kubernetes.default.svc:443') \
    .config('spark.kerberos.principal','hdfs/hadoop-master.hadoop-domain.default-tenant.svc.cluster.local@EXAMPLE.COM') \
    .getOrCreate()


In [ ]:
v3io_path = os.path.join(v3io_fs,'examples','demo.csv')
print(v3io_path)

v3io_df = spark.read.csv(v3io_path)
v3io_df.show()

In [ ]:
output_path = os.path.join(hdfs_fs,'user/hdfs/output.csv')
print(output_path)

v3io_df.write.csv(output_path, mode='overwrite')

In [ ]:
hdfs_df = spark.read.csv(output_path)
hdfs_df.show()

In [ ]:
spark.stop()